In [ ]:
##Test fuction to check for faults later
import numpy as np
from matplotlib.transforms import Affine2D
import matplotlib.pyplot as plt
from PIL import Image
import os
import scipy.ndimage as nd

def test_shapes(arr:np.array):
    if not len(arr.shape) == 3:              # testing (n,m,l), whereas the next tests if l=3
        raise ValueError(f'Your bitmap is not on the form (n,m,l), but {arr.shape}')
    
    if not arr.shape[-1]  == 3:              # testing (n,m,3)
        raise ValueError(f'Your bitmap is not on the form (n,m,3), but {arr.shape}')


def test_rgb_identical(arr:np.array):
    for i in range(1,arr.shape[-1]):
        if np.all(arr[:,:,0] == arr[:,:,i]):
            pass
        else:
            print('Warning: Not all RGB color channels are identical')

def test_max_min(arr:np.array):
    if not arr.max() == 255:
        print(f'Warning: Maximum value is not 255 but {arr.max()}')
    if not arr.min() == 0:
        print(f'Warning: Minimum value is not 0 but {arr.min()}')

    if arr.max() > 255:
        raise ValueError('Max value exceeding limits')
    if arr.min() < 0:
        raise ValueError('Minimum value less than 0')

def is_binary(arr:np.array):
    if not np.all((arr == 0) | (arr == 255)):
        print('FYI: This is not a binary bitmap. This is not a problem, but some pixels will be milled/deposited with a smaller dwell time. If you want to make it binary, try to threshold it.')

def test_dtype(arr: np.array):
    if arr.dtype != np.uint8:
        raise TypeError('dtype is not np.uint8!')
    

def run_tests(arr:np.array):
    """Takes in the *.bmp file as a numpy array and test a couple of formalities."""
    test_max_min(arr)
    test_rgb_identical(arr)
    is_binary(arr)
    test_dtype(arr)

In [ ]:
# Make single triangle
# Create a custom polygon shape
def draw_custom_shape(ax,translation=(0, 0),color='black',tykkelse_mellom = 2,tykkelse_øy = 2,vinkel_s_form = np.pi/6, vinkel_ende = np.pi/3):
    'Define the shape of the repeating pattern'
    endeform = tykkelse_øy/np.tan(vinkel_ende)
    endelengde = 2*tykkelse_øy
    hypotus = tykkelse_øy + tykkelse_øy/np.tan(np.pi/6)
    
    p1,p2,p3 = (tykkelse_øy, -endeform), (tykkelse_øy, -endeform - endelengde), (tykkelse_øy-hypotus*np.cos(vinkel_s_form), (-endeform - endelengde) -hypotus*np.sin(vinkel_s_form))
    p4,p5,p6 =  (p3[0], p3[1]-endelengde), (p3[0]-tykkelse_øy, p3[1]-endelengde+endeform), (p3[0]-tykkelse_øy, p3[1]+endeform)
    p7 = (p6[0]+hypotus*np.cos(vinkel_s_form), p6[1]+hypotus*np.sin(vinkel_s_form))

    points = np.array([
        [0, 0],     #nullpunkt/startpunkt
        [p1[0],p1[1]],
        [p2[0],p2[1]],
        [p3[0],p3[1]],
        [p4[0],p4[1]],
        [p5[0],p5[1]],
        [p6[0],p6[1]],
        [p7[0],p7[1]],
        [0, 0]  #endepunkt
    ])
    
    #1 Apply rotation and translation of first and Create a polygon from the points
    transform1 = Affine2D().rotate_deg(0).translate(*translation)
    points1 = transform1.transform(points)
    
    #2 Apply rotation and translation of second and Create a polygon from the points
    transform2 = Affine2D().rotate_deg(300).translate(*(translation[0] - tykkelse_mellom,translation[1]))
    points2 = transform2.transform(points)

    #3 Apply rotation and translation of third and Create a polygon from the points
    transform3 = Affine2D().rotate_deg(240).translate(*(translation[0]+p4[0] - tykkelse_mellom*np.sin(np.pi/6),translation[1]+p4[1] - tykkelse_mellom*np.cos(np.pi/6)))
    points3 = transform3.transform(points)
    for po in [points1,points2,points3]:
        x,y = [i[0] for i in po],[x[1] for x in po]
        ax.fill(x,y)

def create_pattern(a=1,b=1,c=np.pi/3,size=(100,100)):
    # Set figure size to 8x6 inches (800x600 pixels at 100 DPI)
    fig, ax = plt.subplots(figsize=size, dpi=1)
    draw_custom_shape(ax,tykkelse_mellom = a,tykkelse_øy = b,vinkel_s_form=c)
    ax.set_aspect('equal')
    ax.axis('off')
    png_filename = 'sun_pattern.png'
    plt.savefig(png_filename, format='png')
    
        # Loading image.
    fname = 'sun_pattern.png'                             ## You can change this
    img = plt.imread(fname) 

    # Convert to shape (n,m)
    img = img[:,:,:3].mean(axis=-1)                 # Grayscale by taking the mean over RGB
    # img = img[:,:,0]                              # Grayscale by picking the red color channel, 1 is green and 2 is blue (3 is alpha - transparency)


    ### Optional: Image processing
    ## Shift
    img -= img.min()
    ## Normalize
    img /= img.max()

    ## Thresholding (makes img binary)
    th  = 0.5                                     ## Feel free to change this value
    img = img > th

    # creating an empty (n,m,3) array and fill with values
    img1 = np.zeros((img.shape[0], img.shape[1], 3))   
    img1[:,:,0] = img
    img1[:,:,1] = img
    img1[:,:,2] = img

    # Convert to PIL.Image
    pilimg = Image.fromarray(np.uint8(img1*255))     
    # Save the file, remember to upload to NanoNet
    
    os.makedirs('bmp_folder_form', exist_ok=True)
    fname = f'_WidthB{a}_WidthI{b}_Angle{np.round(c,2)}_Size{size}.bmp'
    file_path = os.path.join('bmp_folder_form', fname)
    pilimg.save(file_path)


In [ ]:
###makes the pattern from the shape
def remove_padding_all_sides(origin_image):
    depadded_image = origin_image
    remove_next_line = (depadded_image[:1,:].max() == 0)
    while remove_next_line:
        tmp = depadded_image[1:,:]
        depadded_image = tmp

        remove_next_line = (depadded_image[:1,:].max() == 0)
    

    remove_next_line = (depadded_image[-1:,:].max() == 0)
    while remove_next_line:
        tmp = depadded_image[:-1,:]
        depadded_image = tmp

        remove_next_line = (depadded_image[-1:,:].max() == 0)
    
    remove_next_line = (depadded_image[:,:1].max() == 0)
    while remove_next_line:
        tmp = depadded_image[:,1:]
        depadded_image = tmp

        remove_next_line = (depadded_image[:,:1].max() == 0)
    
    remove_next_line = (depadded_image[:,-1:].max() == 0)
    while remove_next_line:
        tmp = depadded_image[:,:-1]
        depadded_image = tmp

        remove_next_line = (depadded_image[:,-1:].max() == 0)
    
    return depadded_image

def normaize_img(to_normalize):
    tmp = to_normalize
    tmp -= tmp.min()
    tmp /= tmp.max()
    return tmp

def invert_img(to_invert):
    tmp = to_invert
    tmp = -1*(tmp - tmp.max())
    return tmp

def tile_bmp_from_origin_stencil (nameofbmp_origin_island, final_image_dims, base_offsets_hv, spacing_squeues_hv, spaceing_mults_hv, save_bmp = True, remove_padding_from_origin=True, show_image_popups=False):

    horisontal_base_offset = base_offsets_hv[0]
    vertical_base_offset = base_offsets_hv[1]


    horisontal_spacing_squeuer = spacing_squeues_hv[0]
    vertical_spacing_squeuer = spacing_squeues_hv[1]

    horisontal_spaceing_mult = spaceing_mults_hv[0]
    vertical_spaceing_mult = spaceing_mults_hv[1]

    origin_island = plt.imread(nameofbmp_origin_island)[:,:,:3].mean(axis=-1)


    origin_island = normaize_img(origin_island)


    inverted_origin_island = invert_img(origin_island)

    stencil = inverted_origin_island

    if remove_padding_from_origin:
        stencil = remove_padding_all_sides(stencil)

    if show_image_popups:
        plt.imshow(stencil)
        plt.show()

    grid_points = np.zeros(final_image_dims)

    horisontal_total_offset = horisontal_base_offset + stencil.shape[0]//2 
    vertical_total_offset = vertical_base_offset + stencil.shape[0]//2 

    horisontal_spaceing = int((stencil.shape[1]*2 + horisontal_spacing_squeuer)*horisontal_spaceing_mult)
    vertical_spacing = int((stencil.shape[0] + vertical_spacing_squeuer)*vertical_spaceing_mult)

    grid_points[vertical_total_offset:-vertical_total_offset:vertical_spacing    ,   horisontal_total_offset:-horisontal_total_offset:horisontal_spaceing] = 1

    grid_points[vertical_total_offset +vertical_spacing//2:-vertical_total_offset :vertical_spacing    ,   horisontal_total_offset +horisontal_spaceing//2:-horisontal_total_offset:horisontal_spaceing] = 1


    combined_grid_inverted = nd.convolve(grid_points, stencil, mode='constant')

    if show_image_popups:
        plt.imshow(combined_grid_inverted)
        plt.show()

    # creating an empty (n,m,3) array and fill with values
    combined_grid_inverted_to_bmp = np.zeros((combined_grid_inverted.shape[0], combined_grid_inverted.shape[1], 3))   
    combined_grid_inverted_to_bmp[:,:,0] = combined_grid_inverted
    combined_grid_inverted_to_bmp[:,:,1] = combined_grid_inverted
    combined_grid_inverted_to_bmp[:,:,2] = combined_grid_inverted

    combined_grid_normal = invert_img(combined_grid_inverted)

    combined_grid_normal_to_bmp = np.zeros((combined_grid_normal.shape[0], combined_grid_normal.shape[1], 3))   
    combined_grid_normal_to_bmp[:,:,0] = combined_grid_normal
    combined_grid_normal_to_bmp[:,:,1] = combined_grid_normal
    combined_grid_normal_to_bmp[:,:,2] = combined_grid_normal

    name_of_origin_fixed = nameofbmp_origin_island.split('/')[-1]

    base_offset_text = f'b-offset-hv_{horisontal_base_offset}_{vertical_base_offset}'

    space_squeue_text = f's-squeue-hv_{horisontal_spacing_squeuer}_{vertical_spacing_squeuer}'

    space_mult_text = f's-mult-hv_{horisontal_spaceing_mult}_{vertical_spaceing_mult}'


    if save_bmp:
        os.makedirs('bmp_pattern_normal', exist_ok=True)
        nameofbmp_inverted = f'bmp_pattern_normal/000_Inv_{base_offset_text}_{space_squeue_text}_{space_mult_text}_{name_of_origin_fixed}'

        # Convert to PIL.Image
        pilimg = Image.fromarray(np.uint8(combined_grid_normal_to_bmp*255))     
        pilimg.save(nameofbmp_inverted) 

        test_img = plt.imread(nameofbmp_inverted)
        print('Test starting on ', nameofbmp_inverted)
        run_tests(test_img)
        print('Test ended')
        print(test_img.shape,'\n') 

        os.makedirs('bmp_pattern_invert', exist_ok=True)
        nameofbmp_normal = f'bmp_pattern_invert/000_Nor_{base_offset_text}_{space_squeue_text}_{space_mult_text}_{name_of_origin_fixed}'

        # Convert to PIL.Image
        pilimg = Image.fromarray(np.uint8(combined_grid_inverted_to_bmp*255))     
        # Save the file, remember to upload to NanoNet¨
        pilimg.save(nameofbmp_normal) 

        test_img = plt.imread(nameofbmp_normal)
        print('Test starting on ', nameofbmp_normal)
        run_tests(test_img)
        print('Test ended')
        print(test_img.shape,'\n') 

In [ ]:
#Make multiple with different parameters 
tykkelse_mellom = [1,4,8]
tykkelse_oy = [1,4,8,16]
liste_vinkel_s = [np.pi/6,np.pi/4,np.pi/3]
sizes = [20,30,40,60,80,100]


for param1 in tykkelse_oy:
    for param2 in tykkelse_mellom:
            for param3 in liste_vinkel_s:
                for param4 in sizes:
                    create_pattern(a=param2, b=param1,c=param3,size=(param4,param4))
                    plt.close()
                

In [ ]:
#########################    mass produce below     ###################
directory = 'bmp_folder_form'
show_image_popups_tmp = False
output_bmp_tmp = True
remove_padding_tmp = True

final_image_dims_tmp = (512, 512)

base_offsets_hv_tmp = (20,20)
spaceing_squeues_hv_tmp = (-6, 0)
spaceing_mults_hv_tmp = (1, 1)


for filename in os.listdir(directory):      # iterate over files in directory
    f = os.path.join(directory, filename)
    
    # Skip .DS_Store files
    if filename == '.DS_Store':
        continue
    
    # Check if it is a file and process it
    if os.path.isfile(f):
        try:
            tile_bmp_from_origin_stencil(f, final_image_dims_tmp, base_offsets_hv_tmp, spaceing_squeues_hv_tmp, spaceing_mults_hv_tmp, output_bmp_tmp, remove_padding_tmp, show_image_popups_tmp)
        except Exception as e:
            print(f"Error processing {f}: {e}")